# Pytorch for reinforcement learning

Here is teh quick introduction to reinforcement learning with pytorch

In [1]:
import pfrl
import torch
import torch.nn
import gym
import numpy

PFRL can be used for any problems if they are modeled as "enviroments". Open AI gym provides various kinds of benchmark environ ments and defined scommon interface among them.

In [2]:
env = gym.make('CartPole-v0')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

obs = env.reset()
print('initial observation:', obs)

action = env.action_space.sample()
obs, r, done, info = env.step(action)
print('next observation:', obs)
print('reward:', r)
print('done:', done)
print('info:', info)

# Uncomment to open a GUI window rendering the current state of the environment
# env.render()

observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)
initial observation: [-0.04186687  0.02193018  0.04498008  0.01211728]
next observation: [-0.04142827 -0.17380701  0.04522243  0.3186458 ]
reward: 1.0
done: False
info: {}


PFRL provides various agents, each of which implements a deep reinforcement learning aglrithm.

Let's try to use DoubleDQN algorithm which is implemented by `pfrl.agents.DoubleDQN`. this algorithm trains a Q-function that receives an observation and returns an expected future return for each action that agent can take. You an define your Q-function as `torch.nn.Module` as below.

In [3]:
class QFunction(torch.nn.Module):
    def __init__(self, obs_size, n_actions):
        super().__init__()
        self.l1 = torch.nn.Linear(obs_size, 50)
        self.l2 = torch.nn.Linear(50, 50)
        self.l3 = torch.nn.Linear(50, n_actions)
        
    def forward(self, x):
        h = x
        h = torch.nn.functional.relu(self.l1(h))
        h = torch.nn.functional.relu(self.l2(h))
        h = self.l3(h)
        
        return pfrl.action_value.DiscreteActionValue(h)
    
obs_size = env.observation_space.low.size
n_actions = env.action_space.n
q_function = QFunction(obs_size, n_actions)

`pfrl.q_fuctions.DiscrenteActionValueHead` is just a torch.nn.Module that packs ints input to `pfrl.action_value.DiscreteActionValue`

As usual in PyTorch, `torch.optim.Optimizer` is used to optimize a model


In [4]:
# Use Adam to optimize q_func. eps=1e-2 is for stability.
optimizer = torch.optim.Adam(q_function.parameters(), eps=1e-2)

# Create Agent

In [5]:
gamma = 0.9

# use epsilon-greedy for exploration
explorer = pfrl.explorers.ConstantEpsilonGreedy(epsilon=0.3, random_action_func= env.action_space.sample)

# DQN uses experience replay
replay_buffer = pfrl.replay_buffers.ReplayBuffer(capacity=10**6)


# since the observations from CartPole-v0 is numpy.float64 
# whie as pytorch only accepts numpy.float32 by default, specify
# a converter as a feature extractor function phi
phi = lambda x: x.astype(numpy.float32, copy=False)

gpu = -1

agent = pfrl.agents.DoubleDQN(
    q_function ,
    optimizer,
    replay_buffer,
    gamma,
    explorer,
    replay_start_size=500,
    update_interval=1,
    target_update_interval=100,
    phi=phi,
    gpu=gpu
)

Now that you have an agent and an environment, it's time to start reinforcement learning!

During training, two methods of `agent` must be called: `agent.act` and `agent.observe` 

* `agent.act(obs)` takes the curernt observation as input and returns an exploraty action. Once the returned action is processed in env, 

* `agent.observe(obs, reeward, done, reset)` then observes the consequences

- `obs` : next observation
- `reward` : an immediate reward
- `done` : a boolean value set to True if reached a terminal state
- `reset` : a boolean value set to True if an episode is interrupted at a non-terminal state, typically by a time limit


In [6]:
n_episodes = 300
max_episode_len = 200

history = []
for i in range(1 , n_episodes+1):
    obs = env.reset()
    R = 0
    t = 0
    
    while True:
        # Uncomment to watch the behavior in GUI window
        
        action = agent.act(obs)
        obs, reward, done, _ = env.step(action)
        R += reward
        t += 1
        
        reset = t == max_episode_len
        agent.observe(obs, reward, done, reset)
        if done or reset:
            history.append(R)
            break
        
    
    if i %10 == 0:
        print('episode : ',i ,'R: ', R)
        
    if i % 50 == 0:
        print('episode : ', agent.get_statistics())

episode :  10 R:  11.0
episode :  20 R:  9.0
episode :  30 R:  11.0
episode :  40 R:  14.0
episode :  50 R:  9.0
episode :  [('average_q', 0.27249628), ('average_loss', 0.29977213240721645), ('cumulative_steps', 533), ('n_updates', 34), ('rlen', 533)]
episode :  60 R:  8.0


episode :  70 R:  10.0


episode :  80 R:  12.0


episode :  90 R:  14.0


episode :  100 R:  10.0
episode :  [('average_q', 5.164761), ('average_loss', 0.20260722177103163), ('cumulative_steps', 1236), ('n_updates', 737), ('rlen', 1236)]


...

episode :  260 R:  175.0


episode :  270 R:  155.0


episode :  280 R:  183.0


episode :  290 R:  200.0


episode :  300 R:  172.0
episode :  [('average_q', 9.975841), ('average_loss', 0.09845643496839329), ('cumulative_steps', 29750), ('n_updates', 29251), ('rlen', 29750)]


Now you finished the training the Double DQN agent for 300 episodes. How good is th agent now? You can evaluate it using `with agent.eval_mode()` . Exploration such as epsilon-greedy is not used anymore.

In [7]:
with agent.eval_mode():
    for i in range(10):
        obs = env.reset()
        R = 0
        t = 0
        while True:
            action = agent.act(obs)
            obs, r , done , _ = env.step(action)
            
            R += r
            t += 1
            
            reset = t== 200
            agent.observe(obs, r, done, reset)
            if done or reset:
                break
        
        print('evaluation episode: ', i, 'R: ', R)

evaluation episode:  0 R:  193.0
evaluation episode:  1 R:  175.0
evaluation episode:  2 R:  182.0
evaluation episode:  3 R:  200.0
evaluation episode:  4 R:  185.0
evaluation episode:  5 R:  200.0
evaluation episode:  6 R:  193.0
evaluation episode:  7 R:  181.0
evaluation episode:  8 R:  195.0
evaluation episode:  9 R:  200.0


# Finishing up

In [8]:
agent.save('agent')

# Shortcut

In [9]:
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=2000,           # Train the agent for 2000 steps
    eval_n_steps=None,       # We evaluate for episodes, not time
    eval_n_episodes=10,       # 10 episodes are sampled for each evaluation
    train_max_episode_len=200,  # Maximum length of each episode
    eval_interval=1000,   # Evaluate the agent after every 1000 steps
    outdir='result',      # Save everything to 'result' directory
)

outdir:result step:200 episode:0 R:200.0


statistics:[('average_q', 10.050055), ('average_loss', 0.04763803150504828), ('cumulative_steps', 29950), ('n_updates', 29451), ('rlen', 29950)]


outdir:result step:400 episode:1 R:200.0


statistics:[('average_q', 10.003617), ('average_loss', 0.07018168530310503), ('cumulative_steps', 30150), ('n_updates', 29651), ('rlen', 30150)]


outdir:result step:600 episode:2 R:200.0


...

evaluation episode 9 length:200 R:200.0


The best score is updated 156.4 -> 200.0


Saved the agent to result/best


Saved the agent to result/2000_finish


(<pfrl.agents.double_dqn.DoubleDQN at 0x107c994c0>,
 [{'average_q': 10.015636,
   'average_loss': 0.0489872798131546,
   'cumulative_steps': 30907,
   'n_updates': 30408,
   'rlen': 30907,
   'eval_score': 156.4},
  {'average_q': 10.080789,
   'average_loss': 0.067765186370234,
   'cumulative_steps': 31750,
   'n_updates': 31251,
   'rlen': 31750,
   'eval_score': 200.0}])

# Rainbow DQN

In [10]:
from pfrl.q_functions import DistributionalDuelingDQN


In [11]:
n_atoms = 51
v_max = 10
v_min = -10

q_func = q_functions.DistributionalFCStateQFunctionWithDiscreteAction(
        obs_size,
        n_actions,
        n_atoms,
        v_min,
        v_max,
        n_hidden_channels=args.n_hidden_channels,
        n_hidden_layers=args.n_hidden_layers,
)print(q_func)

SyntaxError: invalid syntax (3169043061.py, line 13)